In [1]:
import pandas as pd
import os
import json
import numpy as np
from IPython.display import Markdown, display
import ast
import re
# Cho phép hiển thị toàn bộ nội dung transcript
# pd.set_option('display.max_colwidth', None)

# Explode cột category để mỗi

os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")

In [2]:
raw_df = pd.read_parquet(
    "data/content-features-dataset/content_features-new-with-stats-top50WeeklyVideos.parquet")
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   categories                  3004 non-null   object                          
 1   audience_target             3004 non-null   object                          
 2   body_analysis               3004 non-null   object                          
 3   content_style               3004 non-null   object                          
 4   cta_content                 3004 non-null   object                          
 5   cta_type                    3004 non-null   object                          
 6   emotion_keywords            3004 non-null   object                          
 7   has_cta                     3004 non-null   bool                            
 8   highlight_sentences         3004 non-null   object                  

In [3]:
# features_df['pacing'].astype('str').unique()
str_cols = []
for col in raw_df.columns:
    print(f"{col}: {raw_df[col].apply(type).unique()}")

    str_flag = raw_df[col].apply(lambda x: isinstance(x, str)).any()
    if str_flag:
        str_cols.append(col)

str_cols

categories: [<class 'str'>]
audience_target: [<class 'numpy.ndarray'>]
body_analysis: [<class 'numpy.ndarray'>]
content_style: [<class 'numpy.ndarray'>]
cta_content: [<class 'numpy.ndarray'>]
cta_type: [<class 'numpy.ndarray'>]
emotion_keywords: [<class 'numpy.ndarray'>]
has_cta: [<class 'bool'>]
highlight_sentences: [<class 'numpy.ndarray'>]
hook_content: [<class 'str'>]
hook_type: [<class 'numpy.ndarray'>]
pacing: [<class 'numpy.ndarray'>]
structure_style: [<class 'numpy.ndarray'>]
summarization: [<class 'str'>]
tone_of_voice: [<class 'numpy.ndarray'>]
video.id: [<class 'str'>]
desc: [<class 'str'>]
transcript: [<class 'str'>]
takeaways: [<class 'numpy.ndarray'>]
weekly_score_rank: [<class 'float'>]
isAd: [<class 'bool'>]
video.duration: [<class 'float'>]
statsV2.playCount: [<class 'float'>]
statsV2.diggCount: [<class 'float'>]
statsV2.commentCount: [<class 'float'>]
statsV2.shareCount: [<class 'float'>]
statsV2.collectCount: [<class 'float'>]
authorStats.diggCount: [<class 'float'>]

['categories',
 'hook_content',
 'summarization',
 'video.id',
 'desc',
 'transcript']

In [4]:
df = raw_df.copy()
df = df.rename(columns={"highlight_phrases": "highlight_sentences"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   categories                  3004 non-null   object                          
 1   audience_target             3004 non-null   object                          
 2   body_analysis               3004 non-null   object                          
 3   content_style               3004 non-null   object                          
 4   cta_content                 3004 non-null   object                          
 5   cta_type                    3004 non-null   object                          
 6   emotion_keywords            3004 non-null   object                          
 7   has_cta                     3004 non-null   bool                            
 8   highlight_sentences         3004 non-null   object                  

In [5]:
# @title Concat json files to df
# Đường dẫn folder chứa JSON
json_folder = 'data/transcripts-highlight_phrases-new'

# Danh sách lưu kết quả
json_data_list = []

# Lặp qua tất cả các file JSON trong thư mục
for filename in os.listdir(json_folder):
    if not filename.endswith('.json'):
        continue

    file_path = os.path.join(json_folder, filename)
    # loại bỏ phần .json để lấy video_id
    video_id = os.path.splitext(filename)[0]

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        # loại bỏ ký tự điều khiển
        content = re.sub(r'[\x00-\x1F\x7F]', '', content)
        content = json.loads(content)
        content['video.id'] = video_id  # thêm video.id vào để merge
        json_data_list.append(content)
    except Exception as e:
        # print(f"❌ Lỗi khi đọc file {file_path}: {e}")
        continue

# Chuyển thành DataFrame từ danh sách JSON
json_df = pd.DataFrame(json_data_list)
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005 entries, 0 to 3004
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   highlight_phrases  3005 non-null   object
 1   video.id           3005 non-null   object
dtypes: object(2)
memory usage: 47.1+ KB


In [6]:
df = df.merge(json_df, how='left', on='video.id')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   categories                  3004 non-null   object                          
 1   audience_target             3004 non-null   object                          
 2   body_analysis               3004 non-null   object                          
 3   content_style               3004 non-null   object                          
 4   cta_content                 3004 non-null   object                          
 5   cta_type                    3004 non-null   object                          
 6   emotion_keywords            3004 non-null   object                          
 7   has_cta                     3004 non-null   bool                            
 8   highlight_sentences         3004 non-null   object                  

In [8]:
# features_df['pacing'].astype('str').unique()
str_cols = []
for col in df.columns:
    print(f"{col}: {df[col].apply(type).unique()}")

    str_flag = df[col].apply(lambda x: isinstance(x, str)).any()
    if str_flag:
        str_cols.append(col)

str_cols

categories: [<class 'str'>]
audience_target: [<class 'numpy.ndarray'>]
body_analysis: [<class 'numpy.ndarray'>]
content_style: [<class 'numpy.ndarray'>]
cta_content: [<class 'numpy.ndarray'>]
cta_type: [<class 'numpy.ndarray'>]
emotion_keywords: [<class 'numpy.ndarray'>]
has_cta: [<class 'bool'>]
highlight_sentences: [<class 'pandas.core.series.Series'>]
hook_content: [<class 'str'>]
hook_type: [<class 'numpy.ndarray'>]
pacing: [<class 'numpy.ndarray'>]
structure_style: [<class 'numpy.ndarray'>]
summarization: [<class 'str'>]
tone_of_voice: [<class 'numpy.ndarray'>]
video.id: [<class 'str'>]
desc: [<class 'str'>]
transcript: [<class 'str'>]
takeaways: [<class 'numpy.ndarray'>]
weekly_score_rank: [<class 'float'>]
isAd: [<class 'bool'>]
video.duration: [<class 'float'>]
statsV2.playCount: [<class 'float'>]
statsV2.diggCount: [<class 'float'>]
statsV2.commentCount: [<class 'float'>]
statsV2.shareCount: [<class 'float'>]
statsV2.collectCount: [<class 'float'>]
authorStats.diggCount: [<cla

['categories',
 'hook_content',
 'summarization',
 'video.id',
 'desc',
 'transcript']

In [9]:
# df.to_parquet(
#     "data/content-features-dataset/content_features-new-with-stats-top50WeeklyVideos.parquet", index=False)